Most scientific operations can be expressed as map-assemble operations. Or, at least doing the numerical calculations for the PDE discretizations. Perhaps we need to do something more complicated and application specific. Having to copy-and-paste assemble.c for every operation is not viable. There is filter.c, which does a hypergraph filtering operation. Cornflakes offers a Python exposure of the map-assemble loop.

In [1]:
import cornflakes as cf
import numpy as np
import husk_spring

This is the trivial setup from the first example.

In [2]:
x = cf.PP.init_grid(10,10,[0,0],[1,0],[0,1])
y = x.copy()
H = cf.Graphers.Build_Pair_Graph(x,0.2)
dm_scalar = cf.Dofmap_Strided(1)
dm_vector = cf.Dofmap_Strided(2)
data = {
    'x':(x,dm_vector),
    'y':(y,dm_vector),
    'k':(np.array([1.0]),dm_scalar)
}

The pieces to the Assemble operation are exposed in the Python interface. It's way slower than calling Assemble, but it lets you define custom operations on the map-assemble framework. Iteration over the Hypergraph will give you the processor's partition of it. It's then your responsibility to make sure the operations you do run in parallel.

In [3]:
for e in H:
    kerin = cf.Assemble.Collect2(husk_spring.kernel_spring_force,e,data)
    kerout = cf.cflib.call_kernel(husk_spring.kernel_spring_force,len(e),kerin)
    # You can do whatever you want here right now!
    # E.g., maybe you want to modify the graph here. The hypergraph 
    # [will] support parallel post-and-commit style operations
    print kerout

[-9. -0.  9.  0. -0.  0.  0.  0.  9.  0. -9. -0.  0.  0. -0.  0.  0.  0.
 -0. -0.]
[ 0. -0.  0.  0. -0. -9.  0.  9.  0.  0.  0. -0.  0.  9. -0. -9.  0.  0.
 -0. -0.]
[-3.18198052 -3.18198052  3.18198052  3.18198052 -3.18198052 -3.18198052
  3.18198052  3.18198052  3.18198052  3.18198052 -3.18198052 -3.18198052
  3.18198052  3.18198052 -3.18198052 -3.18198052  0.          0.         -0.
 -0.        ]
[-3.18198052  3.18198052  3.18198052 -3.18198052  3.18198052 -3.18198052
 -3.18198052  3.18198052  3.18198052 -3.18198052 -3.18198052  3.18198052
 -3.18198052  3.18198052  3.18198052 -3.18198052 -0.          0.          0.
 -0.        ]
[-3.18198052 -3.18198052  3.18198052  3.18198052 -3.18198052 -3.18198052
  3.18198052  3.18198052  3.18198052  3.18198052 -3.18198052 -3.18198052
  3.18198052  3.18198052 -3.18198052 -3.18198052  0.          0.         -0.
 -0.        ]
[ 0. -0.  0.  0. -0. -9.  0.  9.  0.  0.  0. -0.  0.  9. -0. -9.  0.  0.
 -0. -0.]
[-9. -0.  9.  0. -0.  0.  0.  0.  9.  0.